In [1]:
# set up autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
from dataset import DataManager, FrameDataGenerator

# set up data generator
data_manager = DataManager(
        data_path=r"/home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data",
        train_val_test_split=(0.80,0.1,0.1))

data_manager_small = DataManager(
        data_path=r"/home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data_Test",
        train_val_test_split=(1.0,0.0,0.0))

2023-11-24 17:05:41.087586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 17:05:42.159530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-24 17:05:42.970 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-06_09-22-28.avi
2023-11-24 17:05:42.971 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-12_09-53-23.avi
2023-11-24 17:05:42.972 | INFO     | dataset:__init__:29 - Found video file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-06_09-42-29.avi
2023-11-24 17:05:42.973 | INFO     | dataset:__i

In [3]:
# set up tf database
config = {}
config["fold_n_frames"] = 4 # fold n frames to predict the n+1 frame

# train/val/test data loader
train_loader = FrameDataGenerator(file_list=data_manager.get_training_files(),
                                    config=config)
val_loader = FrameDataGenerator(file_list=data_manager.get_validation_files(),
                                    config=config)
test_loader = FrameDataGenerator(file_list=data_manager.get_test_files(),
                                    config=config)
train_loader_small = FrameDataGenerator(file_list=data_manager_small.get_training_files(),
                                    config=config)

# create tensorflow database from the generator
n_col = 7 # the txt files have 7 columns
output_signature = ((tf.TensorSpec(shape = (None, None, 3*config["fold_n_frames"]),
                                    dtype = train_loader.frame_dtype),
                     tf.TensorSpec(shape = (n_col*config["fold_n_frames"],),
                                    dtype = train_loader.action_dtype)),
                    tf.TensorSpec(shape = (None, None, 3),
                                    dtype = train_loader.frame_dtype),
                    )
    
# create train/val/test tf dataset
train_ds = tf.data.Dataset.from_generator(train_loader,
                                          output_signature = output_signature)
val_ds = tf.data.Dataset.from_generator(val_loader,
                                          output_signature = output_signature)
test_ds = tf.data.Dataset.from_generator(test_loader,
                                          output_signature = output_signature)


# set up batch size
train_ds = train_ds.batch(8)
train_ds = train_ds.prefetch(buffer_size = 20)


# for validation and test evaluation, we are gonna sample
# mini batches and calculate the loss, and repeat several
# times and compute the mean loss, as opposed to compute 
# the loss on the whole validation/test set, so we can use
# the same data generation pipeline built for the training
# set
val_ds = val_ds.batch(8)
val_ds = val_ds.prefetch(buffer_size = 20)
test_ds = test_ds.batch(8)
test_ds = test_ds.prefetch(buffer_size = 20)


2023-11-24 17:05:44.395 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-10-24_10-33-15_merge.txt
2023-11-24 17:05:44.396 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-12_09-43-23_merge.txt
2023-11-24 17:05:44.396 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-12_09-28-21_merge.txt
2023-11-24 17:05:44.397 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-11-13_09-22-16_merge.txt
2023-11-24 17:05:44.398 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robotics/ENPM809K_Fundamentals_in_AI_and_DL/Data/Data_2023-10-23_09-40-37_merge.txt
2023-11-24 17:05:44.400 | INFO     | dataset:__init__:127 - Found action file: /home/ji/Dropbox/Robo

In [4]:

train_ds_small = tf.data.Dataset.from_generator(train_loader_small,
                                          output_signature = output_signature)
train_ds_small = train_ds_small.batch(8)
# train_ds_small = train_ds_small.prefetch(1)
train_ds_small = train_ds_small.apply(tf.data.experimental.prefetch_to_device("/gpu:0",buffer_size=1))

In [5]:
import numpy as np

# image in
def outdim(d,f,p,s):
    return np.floor(d+2*p-f)/s+1

# image out size
def outdim_conv2d_tr(d,f,p,s, output_padding):
    return ((d-1)*s+f-2*p+output_padding)

d1 = outdim(192,8,0,2)
d2 = outdim(d1,6,0,2)
d3 = outdim(d2,6,0,2)
d4 = outdim(d3,4,0,2)
print(d4)

d4_ = outdim_conv2d_tr(d4,4,0,2,0)
d3_ = outdim_conv2d_tr(d4_,6,0,2,0)
d2_ = outdim_conv2d_tr(d3_,6,0,2,1)
d1_ = outdim_conv2d_tr(d2_,8,0,2,0)
print(d1_)

9.0
192.0


In [6]:
from model import EncoderNet, DecoderNet, InteractionModule, FramePredictionModel

# some hyper parameters for the network
encoder_output_dim = 1024


img_dim = (192,256,3*config["fold_n_frames"])

encoder = EncoderNet(layer_specs = [
                        {"type":"conv2d","kwargs":{"filters": 64,  "kernel_size": 8, "strides":2,"activation":"relu","input_shape":img_dim}},
                        {"type":"conv2d","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu"}},
                        {"type":"conv2d","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu"}},
                        {"type":"conv2d","kwargs":{"filters": 128, "kernel_size": 4, "strides":2,"activation":"relu"}},
                        {"type":"flatten"},
                        {"type":"dense","kwargs":{"units":encoder_output_dim,"activation":"relu"}}
                        ]
                    )

# get the output size of the last conv layer
conv_out_size = encoder._layers.layers[3].output_shape[1:]

# create the interaction module
interaction_dim, interaction_output_dim = 2048, 2048
interaction = InteractionModule(encoder_dim=encoder_output_dim,
                                action_dim=7,
                                intermediate_dim=interaction_dim,
                                output_dim=interaction_output_dim)

# fc_dim is the dimension of the first fc layer in decoder
nchan = 64
fc_dim = np.prod(conv_out_size[:2])*nchan

decoder = DecoderNet(layer_specs = [
                        {"type":"dense","kwargs":{"units":fc_dim,"activation":"relu","input_shape":(interaction_output_dim,)}},
                        {"type":"reshape","kwargs":{"target_shape":(conv_out_size[0],conv_out_size[1],nchan)}},
                        {"type":"conv2dtr","kwargs":{"filters": 128, "kernel_size": 4, "strides":2,"activation":"relu"}},
                        {"type":"conv2dtr","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu"}},
                        {"type":"conv2dtr","kwargs":{"filters": 128, "kernel_size": 6, "strides":2,"activation":"relu", "output_padding":1}},
                        {"type":"conv2dtr","kwargs":{"filters":   3, "kernel_size": 8, "strides":2,"activation":None}},
                        ]
                    )

# make sure decoder output matches the input image's size
assert(decoder.output_shape[:2]==img_dim[:2])
assert(decoder.output_shape[-1]==3)


In [ ]:
out = next(iter(train_ds))


In [27]:
# create the full model
fpm = FramePredictionModel(encoder=encoder,
                           decoder=decoder,
                           interaction=interaction)

# test with some sample data
(frame_input_sample,actions_sample), frame_output_sample = next(iter(train_ds))
out_test = fpm((frame_input_sample, actions_sample))
assert(out_test.shape==frame_output_sample.shape)

In [ ]:
# try fitting on a small dataset, e.g., a single 5 min video
i = 0
import time
t0 = time.time()
niter = 10
for data in train_ds_small:
    fpm.step(mini_batch=data,verbose=True)
    i+=1
    if i >= niter:
        break
print((time.time()-t0)/niter)

In [ ]:
t0 = time.time()
fpm.step(mini_batch=data,verbose=True)
print(time.time()-t0)

In [ ]:
from PIL import Image
data =frame_output_sample.numpy()[0,:,:,:].squeeze()
data = (data*255).astype(np.uint8)
im = Image.fromarray(data)
im.show()

In [28]:
optimizer = tf.optimizers.Adam(learning_rate=1e-3,clipvalue=0.5)

fpm.compile(optimizer=optimizer,
            loss=tf.keras.losses.MeanSquaredError())



In [ ]:
frame_out = data[1]
tf.reshape(frame_out,(frame_out.shape[0],-1))

In [ ]:
y_true = np.array([[1], [4], [3]], dtype=np.float32)
y_pred = np.array([[2], [4], [4]], dtype=np.float32)
metric = tf.keras.metrics.R2Score()
metric.update_state(y_true, y_pred)
result = metric.result()
result.numpy()


In [54]:
# data = next(iter(train_ds_small))
fpm.train_step(data)

In [23]:
data[1]

<tf.Tensor: shape=(8, 192, 256, 3), dtype=float32, numpy=
array([[[[0.14509805, 0.18039216, 0.22352943],
         [0.14509805, 0.18039216, 0.22352943],
         [0.14509805, 0.18039216, 0.22352943],
         ...,
         [0.20000002, 0.19610909, 0.23532477],
         [0.20114891, 0.19139095, 0.23137257],
         [0.20902269, 0.18157169, 0.22863053]],

        [[0.14509805, 0.18039216, 0.22352943],
         [0.14509805, 0.18039216, 0.22352943],
         [0.14509805, 0.18039216, 0.22352943],
         ...,
         [0.20000002, 0.19610909, 0.23532477],
         [0.20114891, 0.19139095, 0.23137257],
         [0.20902269, 0.18157169, 0.22863053]],

        [[0.14509805, 0.18039216, 0.22352943],
         [0.14509805, 0.18039216, 0.22352943],
         [0.14509805, 0.18039216, 0.22352943],
         ...,
         [0.20000002, 0.19610909, 0.23532477],
         [0.20134439, 0.19158642, 0.23156804],
         [0.21102431, 0.18357334, 0.23063216]],

        ...,

        [[0.05882353, 0.05673996, 

In [58]:
fpm.fit(train_ds_small, batch_size=8, epochs=1, verbose=1)

ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: <tf.Tensor: shape=(), dtype=float32, numpy=0.0>